In [1]:
# importing required modules
from pypdf import PdfReader

# constants
school='ple'
menu_url='https://resources.finalsite.net/images/v1682510818/avongrove/ilhcljkmmiqwx7fgr477/MAYandJUNEPLE2023.pdf'
 
# creating a pdf reader object
reader = PdfReader('may-' + school + '.pdf')
 
# printing number of pages in pdf file
print(len(reader.pages))
 
# getting a specific page from the pdf file
text = ''
for page in reader.pages:
    # extracting text from page
    text = text + page.extract_text()

2


In [2]:
filtered = []
before = []
import sqlite3
import re

# regex patterns
pattern_newlines = re.compile( '\s*\n\s*' )
pattern_extra_spaces = re.compile( '\s\s+' )
pattern_junk = re.compile( 'This\s+institution[\s\S\n]*$')

with sqlite3.connect('lunch.sqlite') as conn:
    insert_sql = 'insert into lunches (school, description, url) values (?,?,?);'

    for line in text.split('Breakfast'):
        # get rid of extra space
        line = re.sub( pattern_newlines, '\n', line)
        
        if '\nLunch\n' in line:
            # get rid of extra space
            line = re.sub( pattern_newlines, '\n', line)
            
            # split on Lunch and only keep the lunch part. chuck the breakfast part
            lunch = line.split('\nLunch\n')[1]
            before.append( line )
            
            # get rid of stuff after the Milk Choice
            lunch = re.sub( pattern_junk, '', lunch)
            line = re.sub( pattern_newlines, '\n', line)


            lunch = lunch.strip()
            conn.execute( insert_sql, (school, line, menu_url ))
            filtered.append(lunch)


In [25]:
import sqlite3

conn = sqlite3.connect('lunch.sqlite')

sql = '''
select * from lunches
'''

conn.execute( sql )

conn.close()

In [47]:
import sqlite3

conn = sqlite3.connect('lunch.sqlite')
print( "Opened database successfully")

sql = '''
create table lunches (
    school text not null,
    datestamp date, 
    description text not null,
    id INTEGER PRIMARY KEY AUTOINCREMENT
);
'''

conn.execute( sql )

print( "Table created successfully")

conn.close()

Opened database successfully
Table created successfully
